[View in Colaboratory](https://colab.research.google.com/github/sungchun12/NSynth-Demos/blob/master/Copy_of_NSynth_Colab.ipynb)

Copyright 2017 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.



# E-Z NSynth

__Neural Audio Synthesis of Musical Notes with WaveNet Autoencoders, ICML 2017__

Jesse Engel, Cinjon Resnick, Adam Roberts, Sander Dieleman, Douglas Eck, Karen Simonyan, Mohammad Norouzi

## Instructions
* This colab notebook has everything you need to upload your own sounds and use NSynth models to reconstruct and interpolate between them.
* You can use these sounds as samples in your own music a la [Andrew Huang Rocking Out with NSynth](https://www.youtube.com/watch?v=AaALLWQmCdI).
* Make sure to use a kernel with a GPU attached by selecting: **Edit >> Notebook Settings >> Hardware Accelerator >> GPU** from the dropdown menu.
* You can run this notebook without writing / seeing a line of code. Just click the triangular **"Play"** button on the left of each cell.
* Start at the top and work your way to the bottom, clicking **"Play"** for each cell.
* If you want to see the code, you can reveal with the menu on the upper right hand side of each cell.


### Other Resources:
* [Nat and Friends "Behind the scenes"](https://www.youtube.com/watch?v=BOoSy-Pg8is)
* [Original Blog Post](https://magenta.tensorflow.org/nsynth)
* [NSynth Instrument](https://magenta.tensorflow.org/nsynth-instrument)
* [Jupyter Notebook Tutorial](https://magenta.tensorflow.org/nsynth-fastgen)
* [ArXiv Paper](https://arxiv.org/abs/1704.01279)
* [Github Code](https://github.com/tensorflow/magenta/tree/master/magenta/models/nsynth)



# Setup Environment

We need to start downloading and choosing a pretrained NSynth model to use. Transfers several GBs, may take a minute or two. Just click Play...

In [2]:
#@title Setup Environment
#@test {"output": "ignore"}


# Install magenta
print('Installing Magenta...\n')
!pip install -q magenta
print('Installing ffmpeg...\n')
!echo "Yes" | apt-get install ffmpeg > /dev/null


print('Downloading Pretrained Models...\n')
# Copy checkpoints from google cloud
# Copying 1GB, takes a minute
print('Getting Instruments Model...\n')
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/nsynth/wavenet-ckpt.tar /content/
print('Getting Voices Model...\n')
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/nsynth/wavenet-voice-ckpt.tar.gz /content/
!cd /content/
!tar -xvf wavenet-ckpt.tar > /dev/null
!tar -xvf wavenet-voice-ckpt.tar.gz > /dev/null


print('Importing Modules...\n')
# Load modules and helper functions
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
%matplotlib inline

from google.colab import files
from magenta.models.nsynth import utils
from magenta.models.nsynth.wavenet import fastgen
from magenta.music.notebook_utils import colab_play as play

def upload(sample_length, sr):
  '''Upload a .wav file.'''
  filemap = files.upload()
  file_list, audio_list = [], []
  for key, value in filemap.iteritems():
    fname = os.path.join('/content/', key)
    with open(fname, 'w') as f:
      f.write(value)
    audio = utils.load_audio(fname, sample_length=sample_length, sr=sr)
    file_list.append(fname)
    audio_list.append(audio)
  return file_list, audio_list

download = files.download

get_name = lambda f: os.path.splitext(os.path.basename(f))[0]

print('Sucess!! Environment is now setup.')


Installing Magenta...

Installing ffmpeg...

Extracting templates from packages: 100%

Getting Instruments Model...

Getting Voices Model...

Importing Modules...

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Sucess!! Environment is now setup.


We have two pretrained models to choose from. One trained on the individual instrument notes of the [NSynth Dataset](https://magenta.tensorflow.org/datasets/nsynth) ("Instruments"), and another trained on a variety of voices in the wild for an art project ("Voices", mixture of singing and speaking). The Instruments model was trained on a larger quantity of data, so tends to generalize a bit better. Neither reconstructs audio perfectly, but both add their own unique character to sounds. Explore them both and see what you like!

In [1]:
#@title Choose a Model { vertical-output: true, run: "auto" }
Model = "Instruments" #@param ["Instruments", "Voices"] {type:"string"}
ckpts = {'Instruments': '/content/wavenet-ckpt/model.ckpt-200000',
         'Voices': '/content/wavenet-voice-ckpt/model.ckpt-200000'}

ckpt_path = ckpts[Model]
print('Using model pretrained on %s.' % Model)

Using model pretrained on Instruments.


# Load Sound Files

Now it's time for you to load your own sound files for processing. You can use either `.wav` files or `.mp3` files, but in either case they will be converted to `.wav`. All files will be downsampled to 16kHz and cropped / silence padded to `Length` seconds as the input and output to the algorithm.

Don't have sounds? You can downloaded free sounds from freesound.org For example:
* https://freesound.org/people/MustardPlug/sounds/395058/
* https://freesound.org/people/cms4f/sounds/159119/
* https://freesound.org/people/juskiddink/sounds/60055/
* https://freesound.org/people/Audeption/sounds/418526/
* https://freesound.org/people/Jagadamba/sounds/255878/


Keep in mind, with the cloud GPU synthesis takes around 4 minutes / 1 second of length.

In [0]:
#@title Set Sound Length (in Seconds) { vertical-output: true, run: "auto" }
Length = 6 #@param {type:"number"}
SR = 16000
SAMPLE_LENGTH = int(SR * Length)

Put all your sound files into a single folder and maker sure to select all the files you want to reconstruct / interpolate between. **(Ctrl/Cmd/Shift Click)**

In [4]:
#@title Upload sound files (.wav, .mp3)

try:
  file_list, audio_list = upload(sample_length=SAMPLE_LENGTH, sr=SR)
  names = [get_name(f) for f in file_list]
  # Pad and peak normalize
  for i in range(len(audio_list)):
    audio_list[i] = audio_list[i] / np.abs(audio_list[i]).max()

    if len(audio_list[i]) < SAMPLE_LENGTH:
      padding = SAMPLE_LENGTH - len(audio_list[i])
      audio_list[i] = np.pad(audio_list[i], (0, padding), 'constant')

  audio_list = np.array(audio_list)
except Exception as e:
  print('Upload Cancelled')
  print(e)

Saving Cello.wav to Cello.wav
Saving Flute.wav to Flute.wav


In [8]:
file_list

[u'/content/Cello.wav', u'/content/Flute.wav']

In [9]:
audio_list

array([[-0.00014499, -0.001606  , -0.00341609, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00513335,  0.0003604 , -0.009982  , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

# Encode

Next we need to encode the audio. This should be relatively fast on a GPU, we will also create interpolations (the midpoints between each encoding) from which to re-synthesize audio.

In [18]:
#@title Generate Encodings
audio = np.array(audio_list)
z = fastgen.encode(audio, ckpt_path, SAMPLE_LENGTH)
print('Encoded %d files' % z.shape[0])


# Start with reconstructions
z_list = [z_ for z_ in z]
name_list = ['recon_' + name_ for name_ in names]

# Add all the mean interpolations
n = len(names)
for i in range(n - 1):
  for j in range(i + 1, n):
    new_z = (z[i] + z[j]) / 2.0
    new_name = 'interp_' + names[i] + '_X_'+ names[j]
    z_list.append(new_z)
    name_list.append(new_name)

print("%d total: %d reconstructions and %d interpolations" % (len(name_list), n, len(name_list) - n))

INFO:tensorflow:Restoring parameters from /content/wavenet-ckpt/model.ckpt-200000
Encoded 2 files
3 total: 2 reconstructions and 1 interpolations


For fun, we can take a look at the encoding of our audio files. They are compressed representations of the audio but have some structure in their own right, (16 numbers, kind of like 16 channels of audio, so there are 16 different lines, colors are arbitrary). 

In [0]:
#@title Visualize Audio and Encoding { vertical-output: true, run: "auto" }
SoundFile = 0 #@param {type:"slider", min:0, max:10, step:1}
file_number = SoundFile

try:
  print(names[file_number])
  play(audio_list[file_number], sample_rate=SR)
  # fig, axs = plt.subplots(2, 1, figsize=(12, 10))
  plt.figure()
  plt.plot(audio_list[file_number])
  plt.title('Audio Signal')

  plt.figure()
  plt.plot(z_list[file_number])
  plt.title('NSynth Encoding')
except Exception as e:
  print e

# Synthesize

On the GPU, this should take about 4 minutes per 1 second of audio per a batch.

In [24]:
#@title Synthesize Interpolations
print('Total Iterations to Complete: %d\n' % SAMPLE_LENGTH)

encodings = np.array(z_list)
save_paths = ['/content/' + name + '.wav' for name in name_list]
fastgen.synthesize(encodings,
                   save_paths=save_paths,
                   checkpoint_path=ckpt_path,
                   samples_per_save=int(SAMPLE_LENGTH / 10))

Total Iterations to Complete: 96000

INFO:tensorflow:Restoring parameters from /content/wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Sample: 0
INFO:tensorflow:Saving: /content/recon_Cello.wav
INFO:tensorflow:Saving: /content/recon_Flute.wav
INFO:tensorflow:Saving: /content/interp_Cello_X_Flute.wav
INFO:tensorflow:Sample: 100
INFO:tensorflow:Sample: 200
INFO:tensorflow:Sample: 300
INFO:tensorflow:Sample: 400
INFO:tensorflow:Sample: 500
INFO:tensorflow:Sample: 600
INFO:tensorflow:Sample: 700
INFO:tensorflow:Sample: 800
INFO:tensorflow:Sample: 900
INFO:tensorflow:Sample: 1000
INFO:tensorflow:Sample: 1100
INFO:tensorflow:Sample: 1200
INFO:tensorflow:Sample: 1300
INFO:tensorflow:Sample: 1400
INFO:tensorflow:Sample: 1500
INFO:tensorflow:Sample: 1600
INFO:tensorflow:Sample: 1700
INFO:tensorflow:Sample: 1800
INFO:tensorflow:Sample: 1900
INFO:tensorflow:Sample: 2000
INFO:tensorflow:Sample: 2100
INFO:tensorflow:Sample: 2200
INFO:tensorflow:Sample: 2300
INFO:tensorflow:Sample: 2400
INFO:tensorf

INFO:tensorflow:Sample: 7300
INFO:tensorflow:Sample: 7400
INFO:tensorflow:Sample: 7500
INFO:tensorflow:Sample: 7600
INFO:tensorflow:Sample: 7700
INFO:tensorflow:Sample: 7800
INFO:tensorflow:Sample: 7900
INFO:tensorflow:Sample: 8000
INFO:tensorflow:Sample: 8100
INFO:tensorflow:Sample: 8200
INFO:tensorflow:Sample: 8300
INFO:tensorflow:Sample: 8400
INFO:tensorflow:Sample: 8500
INFO:tensorflow:Sample: 8600
INFO:tensorflow:Sample: 8700
INFO:tensorflow:Sample: 8800
INFO:tensorflow:Sample: 8900
INFO:tensorflow:Sample: 9000
INFO:tensorflow:Sample: 9100
INFO:tensorflow:Sample: 9200
INFO:tensorflow:Sample: 9300
INFO:tensorflow:Sample: 9400
INFO:tensorflow:Sample: 9500
INFO:tensorflow:Sample: 9600
INFO:tensorflow:Saving: /content/recon_Cello.wav
INFO:tensorflow:Saving: /content/recon_Flute.wav
INFO:tensorflow:Saving: /content/interp_Cello_X_Flute.wav
INFO:tensorflow:Sample: 9700
INFO:tensorflow:Sample: 9800
INFO:tensorflow:Sample: 9900
INFO:tensorflow:Sample: 10000
INFO:tensorflow:Sample: 10100
I

INFO:tensorflow:Sample: 15000
INFO:tensorflow:Sample: 15100
INFO:tensorflow:Sample: 15200
INFO:tensorflow:Sample: 15300
INFO:tensorflow:Sample: 15400
INFO:tensorflow:Sample: 15500
INFO:tensorflow:Sample: 15600
INFO:tensorflow:Sample: 15700
INFO:tensorflow:Sample: 15800
INFO:tensorflow:Sample: 15900
INFO:tensorflow:Sample: 16000
INFO:tensorflow:Sample: 16100
INFO:tensorflow:Sample: 16200
INFO:tensorflow:Sample: 16300
INFO:tensorflow:Sample: 16400
INFO:tensorflow:Sample: 16500
INFO:tensorflow:Sample: 16600
INFO:tensorflow:Sample: 16700
INFO:tensorflow:Sample: 16800
INFO:tensorflow:Sample: 16900
INFO:tensorflow:Sample: 17000
INFO:tensorflow:Sample: 17100
INFO:tensorflow:Sample: 17200
INFO:tensorflow:Sample: 17300
INFO:tensorflow:Sample: 17400
INFO:tensorflow:Sample: 17500
INFO:tensorflow:Sample: 17600
INFO:tensorflow:Sample: 17700
INFO:tensorflow:Sample: 17800
INFO:tensorflow:Sample: 17900
INFO:tensorflow:Sample: 18000
INFO:tensorflow:Sample: 18100
INFO:tensorflow:Sample: 18200
INFO:tenso

INFO:tensorflow:Sample: 22700
INFO:tensorflow:Sample: 22800
INFO:tensorflow:Sample: 22900
INFO:tensorflow:Sample: 23000
INFO:tensorflow:Sample: 23100
INFO:tensorflow:Sample: 23200
INFO:tensorflow:Sample: 23300
INFO:tensorflow:Sample: 23400
INFO:tensorflow:Sample: 23500
INFO:tensorflow:Sample: 23600
INFO:tensorflow:Sample: 23700
INFO:tensorflow:Sample: 23800
INFO:tensorflow:Sample: 23900
INFO:tensorflow:Sample: 24000
INFO:tensorflow:Sample: 24100
INFO:tensorflow:Sample: 24200
INFO:tensorflow:Sample: 24300
INFO:tensorflow:Sample: 24400
INFO:tensorflow:Sample: 24500
INFO:tensorflow:Sample: 24600
INFO:tensorflow:Sample: 24700
INFO:tensorflow:Sample: 24800
INFO:tensorflow:Sample: 24900
INFO:tensorflow:Sample: 25000
INFO:tensorflow:Sample: 25100
INFO:tensorflow:Sample: 25200
INFO:tensorflow:Sample: 25300
INFO:tensorflow:Sample: 25400
INFO:tensorflow:Sample: 25500
INFO:tensorflow:Sample: 25600
INFO:tensorflow:Sample: 25700
INFO:tensorflow:Sample: 25800
INFO:tensorflow:Sample: 25900
INFO:tenso

INFO:tensorflow:Sample: 30400
INFO:tensorflow:Sample: 30500
INFO:tensorflow:Sample: 30600
INFO:tensorflow:Sample: 30700
INFO:tensorflow:Sample: 30800
INFO:tensorflow:Sample: 30900
INFO:tensorflow:Sample: 31000
INFO:tensorflow:Sample: 31100
INFO:tensorflow:Sample: 31200
INFO:tensorflow:Sample: 31300
INFO:tensorflow:Sample: 31400
INFO:tensorflow:Sample: 31500
INFO:tensorflow:Sample: 31600
INFO:tensorflow:Sample: 31700
INFO:tensorflow:Sample: 31800
INFO:tensorflow:Sample: 31900
INFO:tensorflow:Sample: 32000
INFO:tensorflow:Sample: 32100
INFO:tensorflow:Sample: 32200
INFO:tensorflow:Sample: 32300
INFO:tensorflow:Sample: 32400
INFO:tensorflow:Sample: 32500
INFO:tensorflow:Sample: 32600
INFO:tensorflow:Sample: 32700
INFO:tensorflow:Sample: 32800
INFO:tensorflow:Sample: 32900
INFO:tensorflow:Sample: 33000
INFO:tensorflow:Sample: 33100
INFO:tensorflow:Sample: 33200
INFO:tensorflow:Sample: 33300
INFO:tensorflow:Sample: 33400
INFO:tensorflow:Sample: 33500
INFO:tensorflow:Sample: 33600
INFO:tenso

...

INFO:tensorflow:Sample: 53700
INFO:tensorflow:Sample: 53800
INFO:tensorflow:Sample: 53900
INFO:tensorflow:Sample: 54000
INFO:tensorflow:Sample: 54100
INFO:tensorflow:Sample: 54200
INFO:tensorflow:Sample: 54300
INFO:tensorflow:Sample: 54400
INFO:tensorflow:Sample: 54500
INFO:tensorflow:Sample: 54600
INFO:tensorflow:Sample: 54700
INFO:tensorflow:Sample: 54800
INFO:tensorflow:Sample: 54900
INFO:tensorflow:Sample: 55000
INFO:tensorflow:Sample: 55100
INFO:tensorflow:Sample: 55200
INFO:tensorflow:Sample: 55300
INFO:tensorflow:Sample: 55400
INFO:tensorflow:Sample: 55500
INFO:tensorflow:Sample: 55600
INFO:tensorflow:Sample: 55700
INFO:tensorflow:Sample: 55800
INFO:tensorflow:Sample: 55900
INFO:tensorflow:Sample: 56000
INFO:tensorflow:Sample: 56100
INFO:tensorflow:Sample: 56200
INFO:tensorflow:Sample: 56300
INFO:tensorflow:Sample: 56400
INFO:tensorflow:Sample: 56500
INFO:tensorflow:Sample: 56600
INFO:tensorflow:Sample: 56700
INFO:tensorflow:Sample: 56800
INFO:tensorflow:Sample: 56900
INFO:tenso

INFO:tensorflow:Sample: 61400
INFO:tensorflow:Sample: 61500
INFO:tensorflow:Sample: 61600
INFO:tensorflow:Sample: 61700
INFO:tensorflow:Sample: 61800
INFO:tensorflow:Sample: 61900
INFO:tensorflow:Sample: 62000
INFO:tensorflow:Sample: 62100
INFO:tensorflow:Sample: 62200
INFO:tensorflow:Sample: 62300
INFO:tensorflow:Sample: 62400
INFO:tensorflow:Sample: 62500
INFO:tensorflow:Sample: 62600
INFO:tensorflow:Sample: 62700
INFO:tensorflow:Sample: 62800
INFO:tensorflow:Sample: 62900
INFO:tensorflow:Sample: 63000
INFO:tensorflow:Sample: 63100
INFO:tensorflow:Sample: 63200
INFO:tensorflow:Sample: 63300
INFO:tensorflow:Sample: 63400
INFO:tensorflow:Sample: 63500
INFO:tensorflow:Sample: 63600
INFO:tensorflow:Sample: 63700
INFO:tensorflow:Sample: 63800
INFO:tensorflow:Sample: 63900
INFO:tensorflow:Sample: 64000
INFO:tensorflow:Sample: 64100
INFO:tensorflow:Sample: 64200
INFO:tensorflow:Sample: 64300
INFO:tensorflow:Sample: 64400
INFO:tensorflow:Sample: 64500
INFO:tensorflow:Sample: 64600
INFO:tenso

INFO:tensorflow:Sample: 69100
INFO:tensorflow:Sample: 69200
INFO:tensorflow:Sample: 69300
INFO:tensorflow:Sample: 69400
INFO:tensorflow:Sample: 69500
INFO:tensorflow:Sample: 69600
INFO:tensorflow:Sample: 69700
INFO:tensorflow:Sample: 69800
INFO:tensorflow:Sample: 69900
INFO:tensorflow:Sample: 70000
INFO:tensorflow:Sample: 70100
INFO:tensorflow:Sample: 70200
INFO:tensorflow:Sample: 70300
INFO:tensorflow:Sample: 70400
INFO:tensorflow:Sample: 70500
INFO:tensorflow:Sample: 70600
INFO:tensorflow:Sample: 70700
INFO:tensorflow:Sample: 70800
INFO:tensorflow:Sample: 70900
INFO:tensorflow:Sample: 71000
INFO:tensorflow:Sample: 71100
INFO:tensorflow:Sample: 71200
INFO:tensorflow:Sample: 71300
INFO:tensorflow:Sample: 71400
INFO:tensorflow:Sample: 71500
INFO:tensorflow:Sample: 71600
INFO:tensorflow:Sample: 71700
INFO:tensorflow:Sample: 71800
INFO:tensorflow:Sample: 71900
INFO:tensorflow:Sample: 72000
INFO:tensorflow:Sample: 72100
INFO:tensorflow:Sample: 72200
INFO:tensorflow:Sample: 72300
INFO:tenso

INFO:tensorflow:Sample: 76900
INFO:tensorflow:Sample: 77000
INFO:tensorflow:Sample: 77100
INFO:tensorflow:Sample: 77200
INFO:tensorflow:Sample: 77300
INFO:tensorflow:Sample: 77400
INFO:tensorflow:Sample: 77500
INFO:tensorflow:Sample: 77600
INFO:tensorflow:Sample: 77700
INFO:tensorflow:Sample: 77800
INFO:tensorflow:Sample: 77900
INFO:tensorflow:Sample: 78000
INFO:tensorflow:Sample: 78100
INFO:tensorflow:Sample: 78200
INFO:tensorflow:Sample: 78300
INFO:tensorflow:Sample: 78400
INFO:tensorflow:Sample: 78500
INFO:tensorflow:Sample: 78600
INFO:tensorflow:Sample: 78700
INFO:tensorflow:Sample: 78800
INFO:tensorflow:Sample: 78900
INFO:tensorflow:Sample: 79000
INFO:tensorflow:Sample: 79100
INFO:tensorflow:Sample: 79200
INFO:tensorflow:Sample: 79300
INFO:tensorflow:Sample: 79400
INFO:tensorflow:Sample: 79500
INFO:tensorflow:Sample: 79600
INFO:tensorflow:Sample: 79700
INFO:tensorflow:Sample: 79800
INFO:tensorflow:Sample: 79900
INFO:tensorflow:Sample: 80000
INFO:tensorflow:Sample: 80100
INFO:tenso

KeyboardInterrupt: ignored

In [25]:
#@title Download Interpolations
for fname in save_paths:
  print('Downloading: %s' % fname.split('/')[-1])
  download(fname)

Downloading: recon_Cello.wav
Downloading: recon_Flute.wav
Downloading: interp_Cello_X_Flute.wav


In [26]:
#@title Listen to the outputs

print("Originals:\n")
for fname in file_list:
  synth_audio = utils.load_audio(fname,
                                 sample_length=SAMPLE_LENGTH,
                                 sr=SR)
  print(get_name(fname))
  play(synth_audio, sample_rate=SR)

for i, fname in enumerate(save_paths):
  if i == 0:
    print("Reconstructions:\n")
  if i == len(file_list):
    print("Interpolations:\n")
  synth_audio = utils.load_audio(fname,
                                 sample_length=SAMPLE_LENGTH,
                                 sr=SR)
  print(get_name(fname))
  play(synth_audio, sample_rate=SR)

Originals:

Cello


Flute


Reconstructions:

recon_Cello


recon_Flute


Interpolations:

interp_Cello_X_Flute
